In [2]:
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
from pathlib import Path
import re
import shutil
import string

import tensorflow as tf
from tensorflow.keras.utils import text_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, \
    TextVectorization, Embedding, Dropout, GlobalAveragePooling1D, Activation
from tensorflow.keras.losses import BinaryCrossentropy

# Basic Text Classification - part 2
Follow-up from part 1.

## Load data

In [4]:
dataset_dir = Path().home() / r'Desktop\datasets\aclImdb'
train_dir = dataset_dir / 'train'
test_dir = dataset_dir / 'test'

batch_size = 32
seed = 42
raw_train_ds = text_dataset_from_directory(
    directory=train_dir.as_posix(), 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [5]:
raw_val_ds = text_dataset_from_directory(
    directory=train_dir.as_posix(), 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [6]:
raw_test_ds = text_dataset_from_directory(
    directory=test_dir.as_posix(), 
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.


## Prepare for training

In [18]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [19]:
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [20]:
# Make a text-only dataset (without labels)
train_text = raw_train_ds.map(lambda x, y: x)

# Fit the vectorize obj with the tesxts
vectorize_layer.adapt(train_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [21]:
# need to add a dimension to vectorize
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [22]:
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))

# take the first observation
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'Great movie - especially the music - Etta James - "At Last". This speaks volumes when you have finally found that special someone.', shape=(), dtype=string)
Label neg
Vectorized review (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[  86,   17,  260,    2,  222,    1,  571,   31,  229,   11, 2418,
           1,   51,   22,   25,  404,  251,   12,  306,  282,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       

In [23]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print(f'Vocabulary size: {len(vectorize_layer.get_vocabulary())}')

1287 --->  silent
 313 --->  night
Vocabulary size: 10000


In [24]:
# Create the final datasets by
# mapping the TextVectorization layer

train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

## Configure the dataset for performance

In [25]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Create the model

- > The Embedding layer can be understood as a lookup table that maps from integer indices (which stand for specific words) to dense vectors (their embeddings).
- Below, we embed a 10,001-word vocabulary (`max_features` + 1) into 16 dimensions.
- The weight are initially random and are gradually adjusted via backpropagation during training.

In [26]:
embedding_dim = 16


model = tf.keras.Sequential([
    Embedding(max_features + 1, embedding_dim),
    Dropout(0.2),
    GlobalAveragePooling1D(),
    Dropout(0.2),
    Dense(1)])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160016    
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
__________________________________________________

## Loss function and optimizer

- single column contains the output var -> `from_logits=True`
- Classes are ballanced -> accuracy is good performance metric

In [27]:
model.compile(loss=BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

## Train the model

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
625/625 [==============================] - 15s 21ms/step - loss: 0.6621 - binary_accuracy: 0.6999 - val_loss: 0.6121 - val_binary_accuracy: 0.7744
Epoch 2/10
625/625 [==============================] - 5s 8ms/step - loss: 0.5458 - binary_accuracy: 0.8029 - val_loss: 0.4959 - val_binary_accuracy: 0.8252
Epoch 3/10
625/625 [==============================] - 5s 8ms/step - loss: 0.4427 - binary_accuracy: 0.8461 - val_loss: 0.4183 - val_binary_accuracy: 0.8480
Epoch 4/10
625/625 [==============================] - 5s 8ms/step - loss: 0.3763 - binary_accuracy: 0.8665 - val_loss: 0.3724 - val_binary_accuracy: 0.8628
Epoch 5/10
625/625 [==============================] - 5s 8ms/step - loss: 0.3338 - binary_accuracy: 0.8792 - val_loss: 0.3441 - val_binary_accuracy: 0.8680
Epoch 6/10
625/625 [==============================] - 4s 7ms/step - loss: 0.3036 - binary_accuracy: 0.8903 - val_loss: 0.3253 - val_binary_accuracy: 0.8708
Epoch 7/10
625/625 [==============================] - 4s 7ms/s

## Evaluate the model

In [ ]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

## Create a plot of accuracy and loss over time

Training vs validation accuracy:
- The training accuracy peaks earlier.
- After several epochs training gets larger than validation meaning the model over-fits the data. Solution: stop when valid acc no longer increases significantly (early stopping). 

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'b-o', label='Training loss')
plt.plot(epochs, val_loss, 'r-o', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.plot(epochs, acc, 'b-o', label='Training acc')
plt.plot(epochs, val_acc, 'r-o', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()

## Export the model

- Add the preprocess layer to allow to directly predict raw IMDS reviews.

In [ ]:
export_model = Sequential([
  vectorize_layer,
  model,
  Activation('sigmoid')
])

export_model.compile(
    loss=BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

## Heading